In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('clean.csv')
test = pd.read_csv('clean_test.csv')

train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)

train['train']=1
test['train']=0

combined = pd.concat([train,test])

combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
combined['status_group'], labels = pd.factorize(combined['status_group'])

train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

test_df_id = test_df.pop('id')
test_df_statgrp = test_df.pop('status_group')

# train = train_df
train = train_df.sample(frac=0.9)
validate = train_df.sample(frac=0.1)
#test = train_df.sample(frac=0.1)


train_target = train.pop('status_group')
validate_target = validate.pop('status_group')
#test_target = test.pop('status_group')
[xsize, ysize] =train.shape


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD, Adam, RMSprop, Nadam
from keras.regularizers import l2

num_labels = len(np.unique(train_target))
train_target_cat = to_categorical(train_target)
validate_target_cat = to_categorical(validate_target)
#test_target_cat = to_categorical(test_target)

def lr_schedule(epoch):
    lr = 1e-3
    n=60
    if epoch > 4*n:
        lr *= 0.5e-3
    elif epoch > 3*n:
        lr *= 1e-3
    elif epoch > 2*n:
        lr *= 1e-2
    elif epoch > 1*n:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

batch_size = 64
hidden_units = 128
hidden_units2 = 256
hidden_units3 = 512
dropout = 0.1
input_size = (1 * ysize * 1)


kernel_regularizer = l2(0.0001)
model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units2))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units3))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units3))
model.add(Activation('relu'))
model.add(Dense(hidden_units3))
model.add(Activation('relu'))
model.add(Dense(hidden_units3))
model.add(Activation('relu'))
model.add(Dense(hidden_units3))
model.add(Activation('relu'))
model.add(Dense(hidden_units3))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(num_labels))
model.add(Activation('softmax'))


model.summary()

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]
optim = keras.optimizers.Adam(lr=lr_schedule(0), beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000001, amsgrad=True)
# optim = Adam(lr=lr_schedule(0))

model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

# model.fit(train, train_target_cat,  validation_data=(validate, validate_target_cat), epochs=300, batch_size=batch_size, callbacks = callbacks)
model.fit(train, train_target_cat, epochs=300, batch_size=batch_size, callbacks = callbacks)

score = model.evaluate(validate, validate_target_cat, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

test_df_result = model.predict_classes(test_df)
# for i in range(test_df_result.shape[-1]):
#     print(test_df_id[i],labels[test_df_result[i]])

test_df_pred = pd.DataFrame(labels[test_df_result])
test_df_pred.columns = ['status_group']
test_df_id = pd.DataFrame(test_df_id)
test_df_pred = test_df_id.join(test_df_pred)

test_df_pred.to_csv("submission_cooler.csv", index=False)



/home/chico/.virtualenvs/dl4cv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/chico/.virtualenvs/dl4cv/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               2560      
_________________________________________________________________
activation_10 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               33024     
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               131584    
__________

53460/53460 [==============================] - 13s 240us/step - loss: 0.5497 - acc: 0.7668
Epoch 41/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 241us/step - loss: 0.5496 - acc: 0.7666
Epoch 42/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 240us/step - loss: 0.5492 - acc: 0.7672
Epoch 43/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 238us/step - loss: 0.5441 - acc: 0.7691
Epoch 44/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 241us/step - loss: 0.5447 - acc: 0.7689
Epoch 45/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 244us/step - loss: 0.5434 - acc: 0.7687
Epoch 46/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 241us/step - loss: 0.5415 - acc: 0.7697
Epoch 47/300
Learning rate:  0.001
53460/53460 [==============================] - 13s 241us/step - loss: 0.5374 - acc: 0.7709
Epoch 48/300
Learning rate:

53460/53460 [==============================] - 13s 240us/step - loss: 0.4514 - acc: 0.8061
Epoch 105/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 240us/step - loss: 0.4527 - acc: 0.8072
Epoch 106/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 239us/step - loss: 0.4498 - acc: 0.8083
Epoch 107/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 241us/step - loss: 0.4514 - acc: 0.8073
Epoch 108/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 241us/step - loss: 0.4514 - acc: 0.8070
Epoch 109/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 245us/step - loss: 0.4508 - acc: 0.8081
Epoch 110/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 242us/step - loss: 0.4501 - acc: 0.8074
Epoch 111/300
Learning rate:  0.0001
53460/53460 [==============================] - 13s 241us/step - loss: 0.4486 - acc: 0.8086
Epoch 112/300

53460/53460 [==============================] - 13s 238us/step - loss: 0.4349 - acc: 0.8134
Epoch 169/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 239us/step - loss: 0.4365 - acc: 0.8121
Epoch 170/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 239us/step - loss: 0.4391 - acc: 0.8123
Epoch 171/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 239us/step - loss: 0.4382 - acc: 0.8116
Epoch 172/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 239us/step - loss: 0.4372 - acc: 0.8129
Epoch 173/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 240us/step - loss: 0.4367 - acc: 0.8140
Epoch 174/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 240us/step - loss: 0.4364 - acc: 0.8137
Epoch 175/300
Learning rate:  1e-05
53460/53460 [==============================] - 13s 240us/step - loss: 0.4371 - acc: 0.8124
Epoch 176/300
Learni

53460/53460 [==============================] - 13s 238us/step - loss: 0.4370 - acc: 0.8125
Epoch 233/300
Learning rate:  1e-06
53460/53460 [==============================] - 13s 238us/step - loss: 0.4346 - acc: 0.8149
Epoch 234/300
Learning rate:  1e-06
53460/53460 [==============================] - 13s 240us/step - loss: 0.4346 - acc: 0.8136
Epoch 235/300
Learning rate:  1e-06
53460/53460 [==============================] - 13s 239us/step - loss: 0.4384 - acc: 0.8127
Epoch 236/300
Learning rate:  1e-06
53460/53460 [==============================] - 12s 233us/step - loss: 0.4365 - acc: 0.8127
Epoch 237/300
Learning rate:  1e-06
53460/53460 [==============================] - 13s 237us/step - loss: 0.4368 - acc: 0.8139
Epoch 238/300
Learning rate:  1e-06
53460/53460 [==============================] - 13s 235us/step - loss: 0.4358 - acc: 0.8127
Epoch 239/300
Learning rate:  1e-06
53460/53460 [==============================] - 13s 237us/step - loss: 0.4357 - acc: 0.8128
Epoch 240/300
Learni

53460/53460 [==============================] - 13s 239us/step - loss: 0.4352 - acc: 0.8141
Epoch 297/300
Learning rate:  5e-07
53460/53460 [==============================] - 13s 242us/step - loss: 0.4363 - acc: 0.8128
Epoch 298/300
Learning rate:  5e-07
53460/53460 [==============================] - 13s 240us/step - loss: 0.4353 - acc: 0.8137
Epoch 299/300
Learning rate:  5e-07
53460/53460 [==============================] - 13s 244us/step - loss: 0.4363 - acc: 0.8127
Epoch 300/300
Learning rate:  5e-07
5940/5940 [==============================] - 1s 97us/step

Test accuracy: 83.6%
